In [7]:
from pyrfc import Connection,ABAPApplicationError, ABAPRuntimeError, LogonError, CommunicationError
import pandas as pd
import re
import numpy as np

class main():
    def __init__(self):
        # # PRD 100
        # ASHOST='100.100.0.114'
        # CLIENT='100'
        # SYSNR='00'
        # USER='AMMP35'
        # PASSWD='Dua@2022@'
        # ALP 500
        ASHOST='100.100.0.113'
        CLIENT='500'
        SYSNR='50'
        USER='AMMP35'
        PASSWD='Dua@2022@'
        self.conn = Connection(ashost=ASHOST, sysnr=SYSNR, client=CLIENT, user=USER, passwd=PASSWD)

    def query(self, Fields, SQLTable, Where = '', MaxRows=50, FromRow=0):
        """A function to query SAP with RFC_READ_TABLE"""

        # By default, if you send a blank value for fields, you get all of them
        # Therefore, we add a select all option, to better mimic SQL.
        if Fields[0] == '*':
            Fields = ''
        else:
            Fields = [{'FIELDNAME':x} for x in Fields] # Notice the format

        # the WHERE part of the query is called "options"
        options = [{'TEXT': x} for x in Where] # again, notice the format
        print(options)

        # we set a maximum number of rows to return, because it's easy to do and
        # greatly speeds up testing queries.
        rowcount = MaxRows
        try:
        # Here is the call to SAP's RFC_READ_TABLE
            tables = self.conn.call('RFC_READ_TABLE', \
                        QUERY_TABLE = SQLTable, \
                        DELIMITER = '|',\
                        FIELDS = Fields,\
                        OPTIONS = options, \
                        ROWSKIPS = FromRow, ROWCOUNT = MaxRows)
        # We split out fields and fields_name to hold the data and the column names
        except CommunicationError:
            print("Could not connect to server.")
            raise
        except LogonError:
            print("Could not log in. Wrong credentials?")
            raise
        except (ABAPApplicationError, ABAPRuntimeError):
            print("An error occurred.")
            raise
        fields = []
        fields_name = []

        data_fields = tables["DATA"] # pull the data part of the result set
        data_names = tables["FIELDS"] # pull the field name part of the result set

        headers = [x['FIELDNAME'] for x in data_names] # headers extraction
        long_fields = len(data_fields) # data extraction
        long_names = len(data_names) # full headers extraction if you want it

        # now parse the data fields into a list
        for line in range(0, long_fields):
            fields.append(data_fields[line]["WA"].strip())

        # for each line, split the list by the '|' separator
        fields = [x.strip().split('|') for x in fields ]

        # return the 2D list and the headers
        df = pd.DataFrame(fields,columns = headers)
        return df
    
    def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
        '''
        Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
        
        Args:
        - df - dataframe used in pivot table
        - values - values used to aggregrate
        - indices - ordered list of indices to aggregrate by
        - columns - columns to aggregrate by
        - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
        - fill_value - value used to in place of empty cells
        
        Returns:
        -flat table with data aggregrated and tabulated
        
        '''
        listOfTable = []
        for indexNumber in range(len(indices)):
            n = indexNumber+1
            if n == 1:
                table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value,margins=True)
            else:
                table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value)
            table = table.reset_index()
            for column in indices[n:]:
                table[column] = ''
            listOfTable.append(table)
        concatTable = pd.concat(listOfTable).sort_index()
        concatTable = concatTable.set_index(keys=indices)
        return concatTable.sort_index(axis=0,ascending=True)
    # pivot_table_w_subtotals(df=df,values='Value',indices=['Store','Department','Type'],columns=[],aggfunc='sum',fill_value='')


EXPORT MSEG DATA IN PRD.

In [8]:
if __name__ == '__main__':
    # Init the class and connect
    # I find this can be very slow to do...
    rfc = main()
    # Choose your fields and table
    fields = ['MJAHR','MBLNR','EBELN','SHKZG','KZBEW','MATNR','BWART','WERKS','LIFNR','DMBTR','WAERS','MENGE','MEINS','BUDAT_MKPF','CPUDT_MKPF']
    table = 'MSEG'
    # you need to put a where condition in there... could be anything
    # where = ["KZBEW = 'B' AND BWART = '101,102' AND MATNR <> '' AND WERKS = '2100' AND MATNR = '1001010001'"]
    where = ["KZBEW = 'B' AND BWART = '101,102' AND MATNR <> ''"]
    # max number of rows to return
    maxrows = 0

    # starting row to return
    fromrow = 0

    # query SAP
    df_export = rfc.query(fields, table, where, maxrows, fromrow)
    # df_export = df_export.sort_values('ZSEQUEN', ascending= False)\
    # print(max(df_export['MATNR']))
    df_export['ZMONTH'] = df_export['BUDAT_MKPF'].str.slice(4,6)
    df_export['DMBTR'] = df_export.DMBTR.str.replace("*","")
    df_export['DMBTR'] = df_export.DMBTR.astype(float)
    df_export['DMBTR'] = df_export['DMBTR']*100 # VND Currency
    df_export['MENGE'] = df_export.MENGE.astype(float)
    df_export['ZPRICE'] = df_export['DMBTR']/df_export['MENGE']
    # df_export['DMBTR']=df_export['DMBTR'].astype(float)


[{'TEXT': "KZBEW = 'B' AND BWART = '101,102' AND MATNR <> ''"}]


C:\Users\Admin\AppData\Local\Temp\ipykernel_8168\1201460403.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_export['DMBTR'] = df_export.DMBTR.str.replace("*","")


EXPORT VENDOR MASTER DATA

In [9]:
data = "','".join(tuple(set(df_export['LIFNR'])))
field = "PARTNER"
str = ["" + field + " IN ('"  + data +"')"]


In [10]:
if __name__ == '__main__':

    # Init the class and connect
    # I find this can be very slow to do...
    rfc = main()
    # Choose your fields and table
    fields = ['PARTNER','BU_SORT2','BU_GROUP','NAME_ORG1']
    table = 'BUT000'
    # you need to put a where condition in there... could be anything
    # where = ["KZBEW = 'B' AND BWART = '101,102' AND MATNR <> '' AND WERKS = '2100' AND MATNR = '1001010001'"]
    where = [""]

    # where = ["MATKL = '100101'"]
    # max number of rows to return
    maxrows = 0

    # starting row to return
    fromrow = 0

    # query SAP
    df_vendor = rfc.query(fields, table, where, maxrows, fromrow)

[{'TEXT': ''}]


EXPORT MATERIAL DESCRIPTION

In [11]:
if __name__ == '__main__':

    # Init the class and connect
    # I find this can be very slow to do...
    rfc = main()
    # Choose your fields and table
    fields = ['MATNR','MTART','MATKL']
    table = 'MARA'
    # you need to put a where condition in there... could be anything
    # where = ["KZBEW = 'B' AND BWART = '101,102' AND MATNR <> '' AND WERKS = '2100' AND MATNR = '1001010001'"]
    where = [""]

    # where = ["MATKL = '100101'"]
    # max number of rows to return
    maxrows = 0

    # starting row to return
    fromrow = 0

    # query SAP
    df_mara = rfc.query(fields, table, where, maxrows, fromrow)
    print(df_mara)

[{'TEXT': ''}]
                                           MATNR MTART   MATKL
0       TH-007CN00012                             ZHAL    P302
1       TH-008PR00008                             ZHAL    P302
2       TH-010CN00010                             ZHAL    P302
3       TH-020AG0021B                             ZHAL    P302
4       TH-030AG0030B                             ZHAL    P302
...                                          ...   ...     ...
166625  000000002004060180                        ZRO2  200406
166626  000000002004060181                        ZRO2  200406
166627  000000002004060182                        ZRO2  200406
166628  000000002003010056                        ZRO2  200301
166629  000000002003010055                        ZRO2  200301

[166630 rows x 3 columns]


GET DESCRIPTION MATERIAL MASTER DATA.

In [12]:
if __name__ == '__main__':

    # Init the class and connect
    # I find this can be very slow to do...
    rfc = main()
    # Choose your fields and table
    fields = ['MATNR','MAKTG','SPRAS']
    table = 'MAKT'
    # you need to put a where condition in there... could be anything
    # where = ["KZBEW = 'B' AND BWART = '101,102' AND MATNR <> '' AND WERKS = '2100' AND MATNR = '1001010001'"]
    where = ["SPRAS = 'EN'"]

    # where = ["MATKL = '100101'"]
    # max number of rows to return
    maxrows = 0

    # starting row to return
    fromrow = 0

    # query SAP
    df_makt = rfc.query(fields, table, where, maxrows, fromrow)
    print(df_makt)

[{'TEXT': "SPRAS = 'EN'"}]
                                           MATNR  \
0       000000002007811534                         
1       000000002007811535                         
2       000000002007811536                         
3       000000002007811537                         
4       000000002007811538                         
...                                          ...   
166625  000000002004060180                         
166626  000000002004060181                         
166627  000000002004060182                         
166628  000000002003010055                         
166629  000000002003010056                         

                                           MAKTG SPRAS  
0       3T AL.SHEET REFER TO DRAWING                 E  
1       3T AL.SHEET REFER TO DRAWING                 E  
2       3T AL.SHEET REFER TO DRAWING                 E  
3       3T AL.SHEET REFER TO DRAWING                 E  
4       3T AL.SHEET REFER TO DRAWING                 E  
...   

COMPARE DATAFRAME

In [13]:
df_vendor['LIFNR'] =df_vendor['PARTNER']
# Megre Vendor Infomation
df = pd.merge(
    df_export,
    df_vendor,
    on='LIFNR',
    how='left',)
# Megre Material Master Data
df1 = pd.merge(
    df,
    df_makt,
    on='MATNR',
    how='left',)
df2 = pd.merge(
    df1,
    df_mara,
    on='MATNR',
    how='left',)

PRICE REPORT

In [17]:
import numpy as np
table = pd.pivot_table(df2, values='ZPRICE', index=['WERKS','MTART','MATKL','MATNR','MAKTG','MEINS','BU_GROUP','LIFNR','NAME_ORG1'],
                    columns=['MJAHR','ZMONTH'], aggfunc=np.average)
table.to_excel('Report_Price.xlsx')

AMOUNT REPORT

In [ ]:
import numpy as np
table = pd.pivot_table(df2, values='DMBTR', index=['WERKS','MTART','MATKL','MATNR','MAKTG','MEINS','BU_GROUP','LIFNR','NAME_ORG1'],
                    columns=['MJAHR','ZMONTH'], aggfunc=np.average)
table.to_excel('Report_Amount.xlsx')

In [18]:
def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
        '''
        Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
        
        Args:
        - df - dataframe used in pivot table
        - values - values used to aggregrate
        - indices - ordered list of indices to aggregrate by
        - columns - columns to aggregrate by
        - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
        - fill_value - value used to in place of empty cells
        
        Returns:
        -flat table with data aggregrated and tabulated
        
        '''
        listOfTable = []
        for indexNumber in range(len(indices)):
            n = indexNumber+1
            if n == 1:
                table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value,margins=True)
            else:
                table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value)
            table = table.reset_index()
            if len(indices)-1 == n:
                for column in indices[n:]:
                    table[column] = ''
            listOfTable.append(table)
        concatTable = pd.concat(listOfTable).sort_index()
        concatTable = concatTable.set_index(keys=indices)
        return concatTable.sort_index(axis=0,ascending=True)
    # pivot_table_w_subtotals(df=df,values='Value',indices=['Store','Department','Type'],columns=[],aggfunc='sum',fill_value='')


In [19]:
amount = pivot_table_w_subtotals(df2,'ZPRICE',['WERKS','MTART','MATKL','BU_GROUP'],['MJAHR','ZMONTH'],'sum',"")
amount.to_excel("Report_Amount.xlsx")